# Introduction
Group members(& emails?):

# Table of contents
TODO: titles with internal links to move there

# Decomposition and breakdown

blah blah

# Analysis of data given (records.csv)

# Planning and design of algorithm
insert diagram drawn by Cherry here

## Flowchart and pseudocode

lorem ipsum

# Excel things

In [3]:
'''
file = original excel file
rawFileData = giant list where each item is a row converted into a single string
fileHeader = first row "Tutorial Group,Student ID,School,Name,Gender,CGPA"
fileData = each row in the file turned into a list of cells as strings
studentData = list of dictionaries, each representing a student's data ('student ID', 'TG', 'School', 'Name', 'Gender', 'CGPA')

SyntaxError: EOF while scanning triple-quoted string literal (3703543470.py, line 6)

In [4]:
file = open("records.csv", "r")
rawFileData = file.readlines()
fileData = []
for i in rawFileData:
    i = i.split(",")
    i[5] = i[5].replace('\n', '')
    fileData.append(i)
fileHeader = fileData.pop(0) #list of cells from first line

print(fileData[0])

['G-1', '5002', 'CCDS', 'Aarav Singh', 'Male', '4.02']


In [5]:
#global variables


In [6]:
def initStudents():      #creates studentData, a list of dictionaries, each representing a student
    studentData = []
    studentCount = 0
    for row in fileData:
        studentCount += 1
        if row[4] == 'Male':
            gender = True
        elif row[4] == 'Female':
            gender = False
        studentData.append({'ID': int(row[1]),'TG': int(row[0][2:]), 'School': row[2], 'Name': row[3], 'Gender': gender, 'CGPA': float(row[5])})
    return studentData

In [7]:
studentData = initStudents()
print(studentData[0])
print(len(studentData))

{'ID': 5002, 'TG': 1, 'School': 'CCDS', 'Name': 'Aarav Singh', 'Gender': True, 'CGPA': 4.02}
6000


# Prep for Sorting

In [8]:
# Apply function separately to individual tutorial groups
def sortingPrep (studentData, TG):
    # Sort list in descending order of CGPA
    studentData.sort(key=lambda x: x['CGPA'], reverse=True)

    # Split list into buckets
    partition = [studentData[i:i + n] for i in range(0, len(studentData), n)]

    # for index, bucket in enumerate(partition, start=1):
    #     print(f"\nBucket {index} ({len(bucket)}):")
    #     for s in bucket:
    #         print(f"  {s['Name']}, {s['CGPA']}, TG{s['TG']}")

    return partition

In [87]:
def distribute(studentData, n):
    buckets = sortingPrep(studentData, n) 

    total = len(studentData)
    numGrp = total // n
    r = total % n
    grpSizes = [n + 1 if i < r else n for i in range(numGrp)]
    grps = [[] for _ in range(numGrp)]

    # School priority
    schoolCnt = {}
    for s in studentData:
        school = s['School']
        schoolCnt[school] = schoolCnt.get(school, 0) + 1
    sortedSchools = sorted(schoolCnt.keys(), key=lambda x: schoolCnt[x], reverse=True)

    # Gender ratio
    m = sum(1 for s in studentData if s['Gender'])
    mRatio = m / total

    for school in sortedSchools:
        for bucketInd, bucket in enumerate(buckets):
            schoolStudents = [s for s in bucket if s['School'] == school]
            startInd = bucketInd % numGrp
            for i in range(len(schoolStudents)):
                for j in range(numGrp):
                    grpInd = (startInd + j) % numGrp
                    if len(grps[grpInd]) >= grpSizes[grpInd]:
                        continue
                    mGrp = sum(1 for s in grps[grpInd] if s['Gender'])
                    mExp = mRatio * grpSizes[grpInd]
                    if schoolStudents[i]['Gender'] and mGrp >= mExp+1:
                        continue
                    elif not schoolStudents[i]['Gender'] and len(grps[grpInd]) - mGrp > (1 - mRatio) * grpSizes[grpInd]:
                        continue
                    grps[grpInd].append(schoolStudents[i])
                    break

    remaining = [s for s in studentData if all(s not in g for g in grps)]

    for student in remaining:
        notFull = [i for i in range(numGrp) if len(grps[i]) < grpSizes[i]] # only consider grps with space

        # choose group whose current gender ratio deviates opposite to student's gender
        bestInd = None
        bestDiff = float('-inf')

        for i in notFull:
            group = grps[i]
            if not group: # empty grp so any gender fine
                diff = 0
            else:
                mRem = sum(1 for s in group if s['Gender'])
                currRatio = mRem / len(group)
                # positive diff = given group more suitable for this student's gender
                if student['Gender']:
                    diff = (mRatio - currRatio)
                else:
                    diff = (currRatio - mRatio)
            if diff > bestDiff:
                bestDiff = diff
                bestInd = i

        # Place student into best-fitting group
        if bestInd is not None:
            grps[bestInd].append(student)

    return grps


In [88]:
n = 7

# Apply to each TG 
all_tgs = sorted(set(s['TG'] for s in studentData))
for tg in all_tgs:
    tgStuds = [s for s in studentData if s['TG'] == tg]
    totalStuds = len(tgStuds)

    # Get TG gender ratio
    mCount_tg = sum(1 for s in tgStuds if s['Gender'])
    fCount_tg = totalStuds - mCount_tg
    mRatio_tg = mCount_tg / totalStuds if totalStuds else 0
    fRatio_tg = 1 - mRatio_tg

    print(f"\nTutorial Group {tg}")
    print(f"Total students: {totalStuds}")
    print(f"Gender ratio: {mCount_tg}M / {fCount_tg}F "
          f"({mRatio_tg:.2%} male, {fRatio_tg:.2%} female)")

    # Get TG school ratio
    schoolDistTG = {}
    for s in tgStuds:
        schoolDistTG[s['School']] = schoolDistTG.get(s['School'], 0) + 1
    sortedSchoolsTG = sorted(schoolDistTG.keys(), key=lambda x: schoolDistTG[x], reverse=True)
    schoolSummary = ", ".join(f"{sch}: {schoolDistTG[sch]}" for sch in sortedSchoolsTG)
    print(f"School distribution in TG{tg}: {schoolSummary}")

    # Distribute
    finalGrps = distribute(tgStuds, n)

    # Print results per grp
    totalInGrps = 0
    for i, g in enumerate(finalGrps, start=1):
        mCount = sum(1 for s in g if s['Gender'])
        fCount = len(g) - mCount
        totalInGrps += len(g)
        print(f"\nGroup {i} (size {len(g)}): Males: {mCount}, Females: {fCount}")
        for s in g:
            print(f"  - {s['Name']} ({s['School']}, {'M' if s['Gender'] else 'F'}, CGPA: {s['CGPA']})")

    print(f"\nTotal assigned in TG{tg}: {totalInGrps} / {totalStuds}")



Tutorial Group 1
Total students: 50
Gender ratio: 21M / 29F (42.00% male, 58.00% female)
School distribution in TG1: EEE: 10, CoB (NBS): 8, CCDS: 5, SoH: 5, SSS: 5, MAE: 4, CoE: 3, WKW SCI: 2, SPMS: 2, SBS: 1, ASE: 1, NIE: 1, ADM: 1, CCEB: 1, MSE: 1

Group 1 (size 8): Males: 3, Females: 5
  - Adlan Bin Rahman (EEE, M, CGPA: 4.2)
  - Nisha Das (CoB (NBS), F, CGPA: 4.26)
  - Kathy Lau (SSS, F, CGPA: 4.48)
  - Ananya Ramesh (SoH, M, CGPA: 4.2)
  - Meera Singh (CoE, F, CGPA: 4.22)
  - Sana Jain (SPMS, F, CGPA: 4.22)
  - Layla Torres (ASE, F, CGPA: 4.52)
  - Nguyen Van Sam (NIE, M, CGPA: 3.85)

Group 2 (size 7): Males: 1, Females: 6
  - Xun Wei (EEE, F, CGPA: 4.18)
  - Areeba Khan (CoB (NBS), F, CGPA: 4.19)
  - Harlow Wang (CoB (NBS), F, CGPA: 4.18)
  - Maria Ivanovna (SSS, F, CGPA: 4.2)
  - Priya Singh (SSS, F, CGPA: 4.17)
  - Anthony Liu (WKW SCI, M, CGPA: 4.15)
  - Karen Wong (SBS, F, CGPA: 3.95)

Group 3 (size 7): Males: 4, Females: 3
  - Henry Foster (EEE, M, CGPA: 4.11)
  - Savannah 

# Getting it back into an excel file and things

In [97]:
testfile = open("testrecords.csv", "w")
fileHeader = fileHeader
fileData.insert(0, fileHeader) #puts back header line
fileData[1].append('')
fileData[1].append('AAAAAAAH')
print(fileData[0])
print(fileData[1])
for i in range(len(fileData)):  #for each list, join items by ','. Then prints each line into new excel sheet
    fileData[i] = ','.join(fileData[i])
    print(fileData[i], file = testfile, end = '\n')

['Tutorial Group', 'Student ID', 'School', 'Name', 'Gender', 'CGPA', '', 'AAAAAAAH', '', 'AAAAAAAH']
['Tutorial Group', 'Student ID', 'School', 'Name', 'Gender', 'CGPA', '', 'AAAAAAAH', '', 'AAAAAAAH']


# Algorithm Evaluation

## Basic Factors
No majority of students of the same following factors:
1. School
2. Gender
3. Current CGPA

General flow:
TODO: insert flowchart
- For each tutorial group, check for a single factor first, to find the group with the least diversity
- Then rank them to see with tutorial group has a group with the least diversity

### Finding the diversity of a single factor of a single group
Required data: list of students in the group, required factor of the student
<br>
Output: The number for diversity
<br>
Start with finding the number of occurences of each factor

In [ ]:
# Parameters:
#  factor: string of factor to check, same as the key in the dictionary
#  groupData: data needed as a list of dictionaries
# Return:
#  occurences: dictionary of {factor1: numOccurences, factor2: numOccurences ... } as string:int
def FindOccurencesInGroup(factor, groupData : list) -> dict:
    size = len(groupData)
    occurences = {} # Dictionary to keep track of occurences of each factor
    for i in groupData:
        factorOfStudent = i[factor]
        # If factor already exists in occurences, add 1 to it, otherwise create it and set to 1
        if factorOfStudent in occurences.keys():
            occurences[factorOfStudent] += 1
        else:
            occurences[factorOfStudent] = 1
    return occurences

In [ ]:
testGroupData = [
    {"Student ID": 71, "TG" : "G-1", "School" : "SoH", "Name" : "Savannah Taylor", "Gender" : False, "CGPA" : 4.19},
    {"Student ID": 75, "TG" : "G-1", "School" : "CCDS", "Name" : "Sakina Ahmed", "Gender" : False, "CGPA" : 4.03},
    {"Student ID": 162, "TG" : "G-1", "School" : "SSS", "Name" : "Lila Patel", "Gender" : False, "CGPA" : 4.07},
    {"Student ID": 235, "TG" : "G-1", "School" : "CCDS", "Name" : "Ming Zhang", "Gender" : True, "CGPA" : 4.06},
    {"Student ID": 288, "TG" : "G-1", "School" : "CoB (NBS)", "Name" : "Ajay Verma", "Gender" : True, "CGPA" : 4.01}
]
print(FindOccurencesInGroup("School", testGroupData))
print(FindOccurencesInGroup("Gender", testGroupData))
print(FindOccurencesInGroup("CGPA", testGroupData))

{'SoH': 1, 'CCDS': 2, 'SSS': 1, 'CoB (NBS)': 1}
{False: 3, True: 2}
{4.19: 1, 4.03: 1, 4.07: 1, 4.06: 1, 4.01: 1}


#### Then find the most number of occurences out of the total students

In [ ]:
# Summary: returns the key and value of the key with the largest value
# Parameters:
#  dictionary: the dict to find the max of
# Return:
#  key : list, values : int, where key is the key with highest value
def FindMaxInDict(dictionary: dict) -> (list, int):
    maxKeys = []
    maxVal = 0
    for key, value in dictionary.items():
        # update maxKeys and maxVal whenever a new value >= current maxVal
        if value > maxVal:
            maxKeys = [key]
            maxVal = value
        elif value == maxVal:
            maxKeys.append(key)
    return maxKeys, maxVal

In [ ]:
print(FindMaxInDict(FindOccurencesInGroup("School", testGroupData)))
print(FindMaxInDict(FindOccurencesInGroup("Gender", testGroupData)))
print(FindMaxInDict(FindOccurencesInGroup("CGPA", testGroupData)))

(['CCDS'], 2)
([False], 3)
([4.19, 4.03, 4.07, 4.06, 4.01], 1)


## CGPA case
But this does not work for CGPA, for that we want to find the mean, range, and midrange instead
If we use only mean, a majority of high CGPA with a very low CGPA will result in a medium CGPA, this can be solved by looking at midrange, where a high difference between midrange and mean will show that this is happening
However, even with these two, a group will all medium CGPAs will also result in medium mean, and low midrange difference, therefore we add range to determine that this is happening if the range is small.
In conclusion, the result we are looking for is one with an medium mean, high range, and low difference between midrange and mean.
### Finding mean, midrange, and range

In [ ]:
def Mean(*nums) -> float:
    if len(nums) == 1 and type(nums[0]) is list:
        nums = nums[0]
    return sum(nums) / len(nums)

def Range(*nums) -> float:
    if len(nums) == 1 and type(nums[0]) is list:
        nums = nums[0]
    return max(nums) - min(nums)

def Midrange(*nums) -> float:
    if len(nums) == 1 and type(nums[0]) is list:
        nums = nums[0]
    return (max(nums) + min(nums)) / 2

In [ ]:
testList = [4.19, 4.03, 4.07, 4.06, 4.01]
print(Mean(testList))
print(Range(testList))
print(Midrange(testList))

4.072
0.1800000000000006
4.1


## Calculating the factors for each group

In [76]:
# Parameters:
#  factors: list of strings of factor to check, same as the key in the dictionary
#  groupsDatas: List of datas needed as a list of dictionaries
# Return:
#  leastDiverse: dictionary of {factor1: occurencesPercent, factor2: occurencesPercent ... } as string:int
def FindLeastDiverseInTutorialGroup(factors, *groupsDatas) -> dict:
    if len(groupsDatas) == 1 and type(groupsDatas[0]) is list:
        groupsDatas = groupsDatas[0]

    leastDiverse = {}
    for factor in factors:
        leastDiverseOccurence : float = ("factorResult", 0) # % of highest occurence, i.e. 90% of students are from ccds
        for i in groupsDatas:
            occurence = FindMaxInDict(FindOccurencesInGroup(factor, i))
            currentPercent = occurence[1] / len(i)
            if currentPercent > leastDiverseOccurence[1]:
                leastDiverseOccurence = (occurence[0], currentPercent)
            elif currentPercent == leastDiverseOccurence[1]:
                leastDiverseOccurence = (list(leastDiverseOccurence[0] + occurence[0]), currentPercent)
        leastDiverse[factor] = leastDiverseOccurence
    return leastDiverse

In [96]:
testGroupData = [studentData[0:10], studentData[10:20], studentData[20:30], studentData[30:40], studentData[40:50]]
for i in range(5):
    print(f"test group: {i}")
    PrintDictListNice(testGroupData[i])
    print()
# print(testGroupData)
print(FindLeastDiverseInTutorialGroup(["School", "Gender", "CGPA"], testGroupData))

test group: 0
Student ID: 5002              TG: G-1               School: CCDS              Name: Aarav Singh       Gender: True              CGPA: 4.02              
Student ID: 3838              TG: G-1               School: EEE               Name: Aarti Nair        Gender: False             CGPA: 4.05              
Student ID: 2091              TG: G-1               School: EEE               Name: Adlan Bin Rahman  Gender: True              CGPA: 4.2               
Student ID: 288               TG: G-1               School: CoB (NBS)         Name: Ajay Verma        Gender: True              CGPA: 4.01              
Student ID: 4479              TG: G-1               School: CCDS              Name: Amelia Kim        Gender: False             CGPA: 4.11              
Student ID: 5708              TG: G-1               School: SoH               Name: Ananya Ramesh     Gender: True              CGPA: 4.2               
Student ID: 4563              TG: G-1               School: WKW SCI 

In [ ]:
# Weights
CRITERA_WEIGHTS = {"CGPA": 100, "SCHOOL": 100, "GENDER": 100}

# Graphical plots and things

# Challenges and Issues

# Conclusion

# Debug test things or test cases?
doubt they will give marks for test cases tho, prob just leave this for our own use and remove before submission
Highlight

In [94]:
def PrintDictListNice(dictionaryList, spacing : int = 18):
    for i in dictionaryList:
        for k,v in i.items():
            contentLength = 10
            if type(v) is int:
                contentLength = log10(v) + 1
            else:
                contentLength = len(str(v))
            # elif type(v) is bool:
            #     contentLength = len(str(v))
            print(f"{k}: {v}{' ' * max(spacing - contentLength, 1)}", end = '')
        print()
def log10(num):
    count = 0
    while num >= 10:
        num /= 10
        count += 1
    return count

In [80]:
print(log10(1))
print(log10(20))
print(log10(34))
print(log10(64280))

0
1
1
4
